## Translating deep learning code 
### from Pytorch to TensorFlow 


This notebook contains both Tensorflow and Pytorch versions of some functions from the Paper: ***Variational fair Information bottlekneck***. The paper addresses how to generate fair representations of data, devoid of bias w.r.t sensitive attributes (such as age, race, sex, etc.). 

- The original PyTorch functions were retrieved from the author's GitHub repo, which can be found at: https://github.com/sajadn/Variational-Fair-Information-Bottleneck

- Original Paper : http://www.cs.toronto.edu/~sajadn/sajad_norouzi/CSC2541_report.pdf


In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import torch



In [2]:
print(f"numpy: {np.__version__}")
print(f"Pytorch: {torch.__version__}")
print(f"TensorFlow: {tf.__version__}")
print(f"keras: {tf.keras.__version__}")

numpy: 1.19.2
Pytorch: 1.7.1
TensorFlow: 2.4.1
keras: 2.4.0


### Make some random tensors

In [3]:
#random tensors

#Tensorflow
tf_T = tf.random.uniform([2,5])

# Same tensor to pytorch
pt_T = torch.tensor(tf_T.numpy())

In [4]:
tf_T.shape# check shape of tf tensor

TensorShape([2, 5])

In [5]:
pt_T.size()# Check shape of pt tensor

torch.Size([2, 5])

### Gaussian Entropy

In [6]:
# Pytorch
def entropy_gaussian(mu, sigma, mean=True):
    msigma = sigma.view(sigma.shape[0], -1)
    return torch.mean(0.5*(msigma))

In [7]:
#TensorFlow
def tf_entropy_gaussian(mu, sigma, mean=True):
    msigma = tf.reshape(sigma, (sigma.shape[0], -1))
    return tf.reduce_mean(0.5*msigma)

In [8]:
# Pytorch test
entropy_gaussian(pt_T, pt_T)

tensor(0.2740)

In [9]:
# TensorFlow test
tf_entropy_gaussian(tf_T,tf_T)

<tf.Tensor: shape=(), dtype=float32, numpy=0.2740054>

### Negative log bernoulli

In [10]:
# Pytorch
log_sigmoid = torch.nn.LogSigmoid()
def negative_log_bernoulli(data, mu, mean=True, clamp=True):
    if clamp:
        mu = torch.clamp(mu, min=-9.5, max=9.5)
    mdata = data.view(data.shape[0], -1)
    mmu = mu.view(data.shape[0], -1)
    log_prob_1 = log_sigmoid(mmu)
    log_prob_2 = log_sigmoid(-mmu)
    log_likelihood = -torch.mean((mdata*log_prob_1)+(1-mdata)*log_prob_2)
    return log_likelihood

In [11]:
# TensorFlow
def tf_negative_log_bernoulli(data, mu, mean=True, clamp=True):
    cast_double = lambda x: tf.cast(x, tf.double)
    cast_shape = lambda x, d : tf.reshape(x, (d.shape[0],-1))
    chain_ops = lambda x, d : cast_double(cast_shape(x, d))
    
    if clamp:
        mu = K.clip(mu, -9.5, 9.5)
    mdata = chain_ops(data, data)
    mmu = chain_ops(mu, data)
    
    log_prob_1 = tf.math.log_sigmoid(mmu)
    log_prob_2 = tf.math.log_sigmoid(-mmu)
    log_likelihood = -tf.reduce_mean((mdata*log_prob_1)+(1-mdata)*log_prob_2)
    return log_likelihood



In [12]:
#more random tensors
array = tf.random.normal((2,1))
mu = tf.random.uniform(shape=array.shape)


#convert to pytorch tensors again
array_torch = torch.tensor(array.numpy())
mu_torch = torch.tensor(mu.numpy())

In [13]:
# Pytorch test
negative_log_bernoulli(array_torch, mu_torch) 

tensor(2.3194)

In [14]:
# TensorFlow test
tf_negative_log_bernoulli(array, mu , mean=True, clamp=True)

<tf.Tensor: shape=(), dtype=float64, numpy=2.319354580434556>

### KL Divergence loss

In [15]:
#Pytorch
def KL(mu, log_sigma):
    return 0.5*(-log_sigma + mu**2 + log_sigma.exp()).mean()

#TensorFlow
def tf_KL(mu, log_sigma):
    kl_loss = 0.5 * tf.reduce_mean(( - log_sigma + K.square(mu) + K.exp(log_sigma)))
    return kl_loss

In [16]:
#Pytorch test
KL(array_torch, mu_torch)

tensor(1.8843)

In [17]:
#TensorFlow test
tf_KL(array,mu)

<tf.Tensor: shape=(), dtype=float32, numpy=1.8843173>

### Kernel function

In [18]:
# Pytorch
def kernel(a, b): #N x M, K x M
    dist1 = (a**2).sum(dim=1).unsqueeze(1).expand(-1, b.shape[0]) #N x C
    dist2 = (b**2).sum(dim=1).unsqueeze(0).expand(a.shape[0], -1) #N x C
    dist3 = torch.mm(a, b.transpose(0, 1))
    dist = (dist1 + dist2) - (2 * dist3)
    return torch.mean(torch.exp(-dist))

In [19]:
# TensorFlow
def tf_kernel(a,b):
    dist1 = tf.expand_dims(tf.math.reduce_sum((a**2), axis=1), axis=1) * tf.ones(shape=(1,b.shape[0]))
    dist2 = tf.expand_dims(tf.math.reduce_sum((b**2), axis=1), axis=0)* tf.ones(shape=(a.shape[0], 1))
    dist3 = tf.matmul(a, tf.transpose(b, perm=[1, 0]))
    dist = (dist1 + dist2) - (2 * dist3)
    return tf.reduce_mean(tf.math.exp(-dist))

In [20]:
# And more random tensors
array_pt_1 = torch.normal(mean=0, std=1, size=(4,1))
array_pt_2 = torch.normal(mean=0, std=1, size=(2,1))


In [21]:
# Convert to tensors
array_tf_1, array_tf_2 = [tf.constant(x.numpy()) for x in [array_pt_1, array_pt_2]]

In [22]:
# TensorFlow test
tf_kernel(array_tf_1,array_tf_2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.79150426>

In [23]:
# Pytorch Test
kernel(array_pt_1,array_pt_2)

tensor(0.7915)